In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings 
warnings.filterwarnings('ignore')
import folium
from folium.plugins import HeatMap

In [6]:
crime = pd.read_csv('dataset\\Chicago_Crimes.csv')
crime

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,13439321,JH237424,04/14/2024 12:00:00 AM,040XX S PRAIRIE AVE,0890,THEFT,FROM BUILDING,APARTMENT,False,False,...,3,38.0,06,1178707.0,1878256.0,2024,12/21/2024 03:40:46 PM,41.821236,-87.619921,"(41.821236024, -87.619920712)"
1,13437420,JH234779,04/14/2024 12:00:00 AM,023XX W CERMAK RD,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,COMMERCIAL / BUSINESS OFFICE,False,False,...,25,31.0,26,1161210.0,1889347.0,2024,12/21/2024 03:40:46 PM,41.852052,-87.683801,"(41.852051675, -87.683800849)"
2,13428676,JH224478,04/14/2024 12:00:00 AM,043XX W LE MOYNE ST,0917,MOTOR VEHICLE THEFT,"CYCLE, SCOOTER, BIKE WITH VIN",STREET,False,False,...,36,23.0,07,1146960.0,1909501.0,2024,12/21/2024 03:40:46 PM,41.907640,-87.735587,"(41.907640473, -87.735587478)"
3,13429357,JH225293,04/14/2024 12:00:00 AM,039XX W ADAMS ST,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,True,False,...,28,26.0,15,1150158.0,1898721.0,2024,12/21/2024 03:40:46 PM,41.877997,-87.724121,"(41.877997275, -87.724120826)"
4,13430098,JH226395,04/14/2024 12:00:00 AM,011XX W 112TH PL,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,...,21,75.0,06,1170856.0,1830157.0,2024,12/21/2024 03:40:46 PM,41.689421,-87.650123,"(41.6894214, -87.650123247)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249118,13805239,JJ217509,04/12/2025 12:00:00 AM,029XX W LOGAN BLVD,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,APARTMENT,False,False,...,1,22.0,26,1156478.0,1917149.0,2025,04/19/2025 03:41:24 PM,41.928440,-87.700416,"(41.928439867, -87.700415972)"
249119,13804023,JJ215813,04/12/2025 12:00:00 AM,094XX S HARVARD AVE,0430,BATTERY,AGGRAVATED - OTHER DANGEROUS WEAPON,STREET,False,False,...,9,49.0,04B,1175694.0,1842631.0,2025,04/19/2025 03:41:24 PM,41.723545,-87.632040,"(41.723545182, -87.632039508)"
249120,13803926,JJ215943,04/12/2025 12:00:00 AM,084XX S VINCENNES AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,21,71.0,08B,1173850.0,1848976.0,2025,04/19/2025 03:41:24 PM,41.740998,-87.638606,"(41.74099774, -87.638606337)"
249121,13803475,JJ215338,04/12/2025 12:00:00 AM,050XX S ABERDEEN ST,0530,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,STREET,True,False,...,20,61.0,04A,1169838.0,1871348.0,2025,04/19/2025 03:41:24 PM,41.802477,-87.652657,"(41.802477219, -87.652657244)"


In [7]:
theft_df = crime[crime['Primary Type'] == 'THEFT']

heat_data = theft_df[['Latitude', 'Longitude']].dropna().values.tolist()

map_theft = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

HeatMap(heat_data, radius=8).add_to(map_theft)

map_theft

In [8]:
battery_df = crime[crime['Primary Type'] == 'BATTERY']
heat_data = battery_df[['Latitude', 'Longitude']].dropna().values.tolist()
map_battery = folium.Map(location=[41.8781, -87.6298], zoom_start=11)
HeatMap(heat_data, radius=8).add_to(map_battery)
map_battery

In [10]:
b = crime[(crime['Primary Type'] == 'GAMBLING')&(crime['Year']==2024)]

bdf = b.groupby(['Latitude','Longitude']).size().reset_index(name='incident_count')

bdf['normalized_weight'] = (bdf['incident_count'] - bdf['incident_count'].min()) / \
                            (bdf['incident_count'].max() - bdf['incident_count'].min())

heat_data = bdf[['Latitude','Longitude','incident_count']].values.tolist()

bmap = folium.Map(location=[41.8781, -87.6298], zoom_start=12)

HeatMap(heat_data).add_to(bmap)

# bmap.save('gambling.html')
bmap

In [12]:
other_offense = crime[crime['Primary Type'] == "OTHER OFFENSE"]
other_offense_df = other_offense.groupby(['Longitude','Latitude']).size().reset_index(name='incident_count')
other_offense_df['normalized_weight'] = (other_offense_df['incident_count'] - other_offense_df['incident_count'].min()) / \
                                        (other_offense_df['incident_count'].max() - other_offense_df['incident_count'].min())
other_offense_list = other_offense_df[['Latitude', 'Longitude', 'normalized_weight']].values.tolist()
us = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
HeatMap(other_offense_list).add_to(us)
us.save('other_offense.html')
us

In [15]:
kidnapping = crime[crime['Primary Type'] == "KIDNAPPING"]
kidnapping_df = kidnapping.groupby(['Longitude', 'Latitude']).size().reset_index(name='incident_count')
kidnapping_df['normalized_weight'] = (kidnapping_df['incident_count'] - kidnapping_df['incident_count'].min()) / \
                                      (kidnapping_df['incident_count'].max() - kidnapping_df['incident_count'].min())
kidnapping_list = kidnapping_df[['Latitude', 'Longitude', 'normalized_weight']].dropna().values.tolist()
map0 = folium.Map(location=[41.8781, -87.6298], zoom_start=12)
HeatMap(kidnapping_list).add_to(map0)
map0.save('kidnapping_map.html')
map0